# Fix GT generated with Ilastik

## Set autorelaod

In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
from pathlib import Path
import warnings

import cv2

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm

import ipywidgets as widgets
from IPython.display import Image as IpImage
from IPython.display import display
from ipywidgets import Button, HBox, VBox

import sys  
sys.path.insert(0, os.path.join("..", "scripts"))

import gav_mildiou_func as gof
import gav_mildiou_const as goc
import gav_mildiou_text as got
import gav_mildiou_plotly as gop

import ld_dataset as ldd
import ld_plot as ldp
import ld_image as ldi

warnings.simplefilter("ignore")

## Set paths

In [ ]:
ilastik_masks_path = goc.datain_path.joinpath("images", "ld_sheets", "ilastik_masks")
ilastik_masks_path.is_dir()

In [ ]:
train_masks_path = goc.datain_path.joinpath("images", "ld_sheets", "masks")
train_masks_path.is_dir()

In [ ]:
dst_disks_path = goc.datain_path.joinpath("images", "ld_sheets", "disks")
dst_disks_path.is_dir()

In [ ]:
train_images_path = goc.datain_path.joinpath("images", "ld_sheets", "images")
train_images_path.is_dir()

In [ ]:
train_patches_path = goc.datain_path.joinpath("images", "ld_sheets", "patches")
train_patches_path.is_dir()

## Build  coarse masks list

In [ ]:
src_masks = list(ilastik_masks_path.glob("*.bmp"))
len(src_masks)

## Functions

In [ ]:
def load_mask(file_path: str):
    return cv2.imread(str(file_path), cv2.COLOR_BGR2GRAY)


def load_image(file_path: str):
    return cv2.cvtColor(cv2.imread(str(file_path)), cv2.COLOR_BGR2RGB)


## Build clean masks

### Prototype

#### Load mask

In [ ]:
cb_image = widgets.Dropdown(
    options=sorted([i.name for i in src_masks]),
    description="Select an image:",
    layout=widgets.Layout(width="100%"),
)
disp_image = widgets.Output(layout=widgets.Layout(height="800"))


def on_image_changed(change):
    disp_image.clear_output()
    ldp.visualize_image(
        image=load_mask(file_path=str(ilastik_masks_path.joinpath(change.new))),
        title=change.new,
    )


cb_image.observe(on_image_changed, names="value")

display(VBox([cb_image, disp_image]))


In [ ]:
raw_mask = load_mask(file_path=str(ilastik_masks_path.joinpath(cb_image.value)))
ldp.visualize_image(raw_mask)


In [ ]:
np.unique(raw_mask)

#### Clean ask

In [ ]:
clean_mask = ldi.clean_contours(mask=raw_mask.copy(), size_thrshold=0.75)
ldp.visualize_image(clean_mask)


In [ ]:
np.unique(clean_mask)

In [ ]:
np.count_nonzero(clean_mask)

#### Resize mask

In [ ]:
resized_mask = np.where(
    cv2.resize(
        clean_mask,
        (clean_mask.shape[1] * 2, clean_mask.shape[0] * 2),
        cv2.INTER_NEAREST,
    )
    > 0,
    255,
    0,
).astype(np.uint8)
# resized_mask[resized_mask != 0] == 255
ldp.visualize_image(resized_mask)


In [ ]:
np.unique(resized_mask)

In [ ]:
np.count_nonzero(resized_mask)

In [ ]:
resized_mask.dtype

In [ ]:
np.unique(resized_mask)

#### Do mask in one

In [ ]:
src_image = load_image(train_images_path.joinpath(cb_image.value.split(".")[0] + ".jpg"))


resized_mask = np.where(
    cv2.resize(
        ldi.clean_contours(
            mask=load_mask(
                file_path=str(ilastik_masks_path.joinpath(cb_image.value))
            ).copy(),
            size_thrshold=0.75,
        ),
        (src_image.shape[1], src_image.shape[0]),
        cv2.INTER_NEAREST,
    )
    > 0,
    255,
    0,
).astype(np.uint8)

ldp.visualize_image(resized_mask)


#### Show contours indexes

In [ ]:
contours = ldi.index_contours(resized_mask)

ldp.visualize_image(
    ldi.print_contours_indexes(
        resized_mask,
        contours,
        canvas=ldi.apply_mask(
            load_image(
                file_path=str(
                    train_images_path.joinpath(cb_image.value.split(".")[0] + ".jpg")
                )
            ),
            resized_mask,
            draw_contours=8,
        ),
    ),
    # figsize=(12, 8),
)


In [ ]:
list(contours)

In [ ]:
ld = ldi.get_leaf_disk(
    image=load_image(
        file_path=str(
            train_images_path.joinpath(cb_image.value.split(".")[0] + ".jpg")
        )
    ),
    contours=contours,
    row=contours[0].row,
    col=contours[0].col,
)


ldp.visualize_image(ld)

In [ ]:
ld.shape

In [ ]:
for raw_mask_path in tqdm(src_masks):
    src_image = load_image(
        file_path=train_images_path.joinpath(raw_mask_path.stem + ".jpg")
    )
    cv2.imwrite(
        str(train_masks_path.joinpath(raw_mask_path.stem + ".bmp")),
        np.where(
            cv2.resize(
                ldi.clean_contours(
                    mask=load_mask(file_path=ilastik_masks_path.joinpath(raw_mask_path.stem + ".bmp")),
                    size_thrshold=0.75,
                ),
                (src_image.shape[1], src_image.shape[0]),
                cv2.INTER_NEAREST,
            )
            > 0,
            255,
            0,
        ).astype(np.uint8),
    )


## Build patches

### Load Dataframe

In [ ]:
d = (
    pd.concat(
        [
            pd.read_csv(
                str(
                    Path.cwd().parent.joinpath(
                        goc.dataframes_path,
                        "ld_dataset_ilastik_train.csv",
                    )
                ),
                sep=",",
            ),
            pd.read_csv(
                str(Path.cwd().parent.joinpath(goc.dataframes_path, "ld_dataset.csv")),
                sep=",",
            ),
        ]
    )
    .drop(["Unnamed: 0"], axis=1)
    .drop_duplicates()
    .sort_values(["exp_folder", "experiment", "image_name", "ligne", "colonne"])
    .assign(colonne=lambda x: x.colonne.astype(np.int8))
)
d


In [ ]:
d.hist(figsize=(12, 8));

### Prototype

#### Get sample row

In [ ]:
row = d.sample(n=1)
row

#### Load image and mask

In [ ]:
image_name = row.image_name.to_list()[0]
image_name

In [ ]:
train_images_path.joinpath(image_name + ".jpg")

In [ ]:
image = load_image(train_images_path.joinpath(image_name + ".jpg"))
ldp.visualize_image(image)

In [ ]:
image.shape

In [ ]:
train_masks_path.joinpath(image_name + ".bmp")

In [ ]:
mask = load_mask(train_masks_path.joinpath(image_name + ".bmp"))
ldp.visualize_image(mask)

In [ ]:
mask.shape

In [ ]:
assert(image.shape[0] == mask.shape[0] and image.shape[1] == mask.shape[1])

#### Get leaf disk

In [ ]:
contours = ldi.index_contours(mask)

ldp.visualize_image(
    ldi.print_contours_indexes(
        mask,
        contours,
        canvas=ldi.apply_mask(image.copy(), mask, draw_contours=8),
    ),
)


In [ ]:
ld = ldi.get_leaf_disk_patch(
    image=image,
    mask=mask,
    contours=contours,
    row=row.ligne.to_list()[0],
    col=row.colonne.to_list()[0],
    patch_size=200,
)


ldp.visualize_image(ld)


cv2.imwrite(
    str(
        train_patches_path.joinpath(
            f"{image_name}_{row.ligne.to_list()[0]}_{row.colonne.to_list()[0]}.bmp"
        )
    ),
    cv2.cvtColor(ld, cv2.COLOR_RGB2BGR),
)


In [ ]:
row.colonne.to_list()[0]

### Build patches

In [ ]:
fails = []

for row in tqdm([row for _, row in d.iterrows()]):
    try:
        image_name = row.image_name
        image = load_image(train_images_path.joinpath(image_name + ".jpg"))
        mask = load_mask(train_masks_path.joinpath(image_name + ".bmp"))
        contours = ldi.index_contours(mask)

        cv2.imwrite(
            str(
                train_patches_path.joinpath(
                    f"{image_name}_{row.ligne}_{row.colonne}.bmp"
                )
            ),
            cv2.cvtColor(
                ldi.get_leaf_disk_patch(
                    image=image,
                    mask=mask,
                    contours=contours,
                    row=row.ligne,
                    col=row.colonne,
                    patch_size=200,
                ),
                cv2.COLOR_RGB2BGR,
            ),
        )
    except Exception as e:
        fails.append(f"Failed to extract patch for {image_name}: {row.ligne},{row.colonne}")

fails
